# IBM Watson

Paso 0: Configurar Watson (instalar librerías, armar usuario y conseguir los permisos necesarios).

In [1]:
# !pip install --upgrade watson-developer-cloud
# !pip install --upgrade watson-machine-learning-client

### Configurando la API de watson
En este paso van a precisar usar el api_key que se generan en la página de IBM 

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Eso aparecía a en el notebook que se bajan de la página (ya no funciona):
# from watson_developer_cloud import NaturalLanguageUnderstandingV1
# from watson_developer_cloud.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions

# Esta es la nueva forma de hacerlo
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, EmotionOptions, SentimentOptions

In [3]:
#natural_language_understanding = NaturalLanguageUnderstandingV1(
#    version='2019-07-12',
#    iam_apikey='RqzBP7gaBbeFX2v4gAlUZTw2eBB4w1H9quVVIHl14uRC',
#    url='https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/cf5cd667-3142-4d27-81fc-39e37a150e2a'
#)



In [4]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions

authenticator = IAMAuthenticator('RqzBP7gaBbeFX2v4gAlUZTw2eBB4w1H9quVVIHl14uRC')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)

natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/cf5cd667-3142-4d27-81fc-39e37a150e2a')

### Probando a Watson

Exploremos el tipo de objeto que nos devuelve Watson y como funciona para distintas frases

In [5]:
# Texto objetivo
text = 'This is a very silly text, it drives me crazy when I read it.'

# Importamos el modelo que vamos a usar y le avisamos que features queremo que devuelva
# y en que lenguaje está el texto (esto es mejor que dejarlo adivinar)
response = natural_language_understanding.analyze(text=text,features=Features(emotion=EmotionOptions(), sentiment=SentimentOptions()),language='en')
resultado = response.result
resultado

{'usage': {'text_units': 1, 'text_characters': 61, 'features': 2},
 'sentiment': {'document': {'score': -0.781525, 'label': 'negative'}},
 'language': 'en',
 'emotion': {'document': {'emotion': {'sadness': 0.155847,
    'joy': 0.14858,
    'fear': 0.318921,
    'disgust': 0.126755,
    'anger': 0.494942}}}}

Para tomar sólo alguno de los resultados, podemos acceder al diccionario

In [6]:
print(resultado['sentiment']['document']['score'])

-0.781525


Probamos ahora un texto en español

In [7]:
# Texto objetivo
text = 'Este texto no tiene ningún objetivo, vive en la soledad que genera saber que su existencia no tiene un proposito determinado.'

# Importamos el modelo que vamos a usar y le avisamos que features queremo que devuelva
# y en que lenguaje está el texto (esto es mejor que dejarlo adivinar)
response = natural_language_understanding.analyze(text=text,features=Features(emotion=EmotionOptions(), sentiment=SentimentOptions()),language='es')
resultado = response.result
resultado


{'usage': {'text_units': 1, 'text_characters': 125, 'features': 1},
 'sentiment': {'document': {'score': 0, 'label': 'neutral'}},
 'language': 'es',
 'warnings': ['emotion: unsupported text language: es']}

### Dataset de películas
Este es el dataset propuesto por la plataforma de Acámica, al estár en ingles permite explotar todas las funcionalidades de Watson. Carguen distintos reviews y comprueben si la devolución del analisis de emosiones de Watson coinciden con lo que perciben del texto.

In [8]:
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset_movies/movie_reviews'
movie_reviews = load_files(moviedir, shuffle=True)

In [9]:
def analisis_watson(text):
    text = text.decode("utf-8")
    print(text)
    response = natural_language_understanding.analyze(
        text=text,features=Features(emotion=EmotionOptions(), sentiment=SentimentOptions()))
    return response

In [10]:
print(analisis_watson(movie_reviews.data[4]))
print(movie_reviews.target[4])

kolya is one of the richest films i've seen in some time . 
zdenek sverak plays a confirmed old bachelor ( who's likely to remain so ) , who finds his life as a czech cellist increasingly impacted by the five-year old boy that he's taking care of . 
though it ends rather abruptly-- and i'm whining , 'cause i wanted to spend more time with these characters-- the acting , writing , and production values are as high as , if not higher than , comparable american dramas . 
this father-and-son delight-- sverak also wrote the script , while his son , jan , directed-- won a golden globe for best foreign language film and , a couple days after i saw it , walked away an oscar . 
in czech and russian , with english subtitles . 

{
    "result": {
        "usage": {
            "text_units": 1,
            "text_characters": 727,
            "features": 2
        },
        "sentiment": {
            "document": {
                "score": 0.632957,
                "label": "positive"
            }

### Dataset de Noticias Infobae de hoy

Este dataset se lo preparamos para que puedan explorar cómo funciona Watson en textos en español. Algunas de las funcionalidades se pierden (no reporta las emociones), pero sí clasifica como positivo o negativo los cuerpos de texto. Noten que otra estrategia podría ser traducir el texto y luego aplicarle el algoritmo de Watson en inglés. Esta otra estrategia no siempre funciona bien, depende mucho de la calidad de la traducción.

In [11]:
nombre_archivo = '2019_08_21.feather'

Exploremos el dataset

In [12]:
# Quiero ver el sentimiento de los cuerpos de la noticia.
noticias_hoy = pd.read_feather(nombre_archivo)
noticias_hoy

ImportError: pyarrow is not installed

you can install via conda
conda install pyarrow -c conda-forge
or via pip
pip install -U pyarrow


In [ ]:
cuerpos = noticias_hoy.Texto
titulos = noticias_hoy.Titulo
print(len(cuerpos))

Vamos a calcular el sentimiento para cada una de las noticias y agregarlo al dataframe 

In [ ]:
score_cuerpos = []

for este_texto in cuerpos:
    
    # Quitamos del texto este símbolo que indica final de parrafo
    este_texto = este_texto.replace('\n','')
    
    # Analisis sentimiento del cuerpo
    # Le pedimos solo el sentimiento, y le avisamos que está en español
    response = natural_language_understanding.analyze(text=este_texto,features=Features(sentiment=SentimentOptions()),language='es')
    resultado = response.result
    score_cuerpos.append(resultado['sentiment']['document']['score'])

In [ ]:
print(score_cuerpos)

In [ ]:
# Finalmente agregamos la columna al dataset
noticias_hoy['Sentimiento'] = score_cuerpos
noticias_hoy

Exploremos un poco los resultados. Veamos el promedio de sentimiento de todas las noticias (esto nos va a dar una idea del ánimo general del día). Luego busquemos la noticia más positiva y la más negativa, para ver si coincidimos con el criterio.

In [ ]:
todos_los_valores = noticias_hoy['Sentimiento']
animo_general = np.mean(todos_los_valores)
print(animo_general)

In [ ]:
noticia_positiva = noticias_hoy.loc[noticias_hoy['Sentimiento'].idxmax()]
print(noticia_positiva.Titulo)
print(noticia_positiva.Sentimiento)
#print(noticia_positiva.Subtitulo)
print(noticia_positiva.Texto)

In [ ]:
noticia_negativa = noticias_hoy.loc[noticias_hoy['Sentimiento'].idxmin()]
print(noticia_negativa.Titulo)
print(noticia_negativa.Sentimiento)
#print(noticia_negativa.Subtitulo)
#print(noticia_negativa.Texto)